<a href="https://colab.research.google.com/github/MorganChidley/Final-Year-Project/blob/main/ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the dataset
file_path = "MyDataSET.csv"
data = pd.read_csv(file_path)

# Display the first few rows of the data
data.head()



In [ ]:
import pandas as pd

# Load the dataset
file_path = "MyDataSET.csv"
data= pd.read_csv(file_path)

# Get the shape of the data (number of rows and columns)
data.shape

In [ ]:
import pandas as pd

# Load the dataset
file_path = "MyDataSET.csv"
data= pd.read_csv(file_path)

# Check for missing values
data.isnull().sum()

In [31]:
import pandas as pd

# Load the dataset
file_path = "MyDataSET.csv"
data= pd.read_csv(file_path)

# Remove duplicate rows
data.drop_duplicates(inplace=True)

# Check the new shape of the dataset
data.shape



(903, 2)

In [53]:
import pandas as pd
import re
from urllib.parse import urlparse

# Feature extraction functions

def url_length(url):
    """Returns the length of the URL."""
    return len(url)

def count_special_chars(url):
    """Counts occurrences of special characters."""
    special_chars = ["@", "?", "-", "=", "_", "&", "%", "/", ".", "#"]
    return {char: url.count(char) for char in special_chars}

def count_subdomains(url):
    """Counts the number of subdomains."""
    parsed_url = urlparse(url).netloc
    return parsed_url.count('.')  # Number of dots in domain

def has_https(url):
    """Checks if the URL uses HTTPS."""
    return 1 if urlparse(url).scheme == "https" else 0

# Apply feature extraction

data["url_length"] = data["URL"].apply(url_length)
data["subdomain_count"] = data["URL"].apply(count_subdomains)
data["https"] = data["URL"].apply(has_https)

# Extract special character counts only once
if not all(char in data.columns for char in ["@", "?", "-", "=", "_", "&", "%", "/", ".", "#"]):
    special_chars_df = data["URL"].apply(count_special_chars).apply(pd.Series)
    data = pd.concat([data, special_chars_df], axis=1)

# Remove duplicate columns (keeping the first occurrence)
data = data.loc[:, ~data.columns.duplicated()]

# Display first few rows with new features
data.head()

,URL,ClassLabel,url_length,subdomain_count,https,@,?,-,=,_,&,%,/,.,#,creation_date,expiration_date
0,https://keraekken-loagginnusa.godaddysites.com/,0,47,2,1,0,0,1,0,0,0,0,3,2,0,None,None
1,https://metamsk01lgiix.godaddysites.com/,0,40,2,1,0,0,0,0,0,0,0,3,2,0,None,None
2,http://myglobaltech.in/,0,23,1,0,0,0,0,0,0,0,0,3,1,0,None,None
3,http://djtool-for-spotify.com/,0,30,1,0,0,0,2,0,0,0,0,3,1,0,None,None
4,https://scearmcoommunnlty.com/invent/freind/get,0,47,1,1,0,0,0,0,0,0,0,5,1,0,None,None


In [54]:
data.to_csv("modified_dataset.csv", index=False)